<a href="https://colab.research.google.com/github/batuhanyndny/notebooks/blob/master/fer_2013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir /root/.kaggle/
!touch /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json
!echo '{"username":"batuhanyndny","key":"key"}' > /root/.kaggle/kaggle.json

In [ ]:
!mkdir New

In [ ]:
cd New/

/content/New


In [ ]:
!ls

In [ ]:
!kaggle datasets download -d shawon10/ckplus

  0% 0.00/3.63M [00:00<?, ?B/s]
100% 3.63M/3.63M [00:00<00:00, 120MB/s]


In [ ]:
!ls

ckplus.zip


In [ ]:
import zipfile

with zipfile.ZipFile("ckplus.zip","r") as ref:
  ref.extractall()

In [ ]:
!ls

ck  CK+48  ckplus.zip


In [ ]:
cd CK+48

/content/New/CK+48


In [ ]:
!ls

anger  contempt  disgust  fear	happy  sadness	surprise


In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
os.listdir()

['disgust', 'happy', 'surprise', 'fear', 'sadness', 'anger', 'contempt']

# **Preprocessing**

In [ ]:
paths = os.listdir()
import cv2
import imageio

images = []
index = 0
for path in paths:
  print("Loading ",path)
  for image in os.listdir("{}/".format(path)):
    print(image)
    img = imageio.imread(image)
    img = np.asarray(img)
    input_img_resize = np.resize(input_img,(48,48))
    images.append(input_img_resize)
    index += 1
  print("size of ",path," is ",index)

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        #input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        input_img_resize=cv2.resize(input_img,(48,48))
        img_data_list.append(input_img_resize)

img_data = np.array(images)
img_data = img_data.astype('float32')
img_data = img_data/255
img_data.shape


Loading  disgust
S035_005_00000018.png


FileNotFoundError: ignored

In [ ]:
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:177]=0 #disgust
labels[178:384]=1 #happy
labels[385:633]=2 #surprise
labels[634:708]=3 #fear
labels[709:792]=4 #sadness
labels[793:927]=5 #anger
labels[928:981]=6 #contempt

names = ['disgust','happy','surprise','fear','sadness','anger','contempt']

def getLabel(id):
    return ['disgust','happy','surprise','fear','sadness','anger','contempt'][id]

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

Y = np_utils.to_categorical(labels, num_classes)

#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=42)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=2)

x_test=X_test
print(X_train.shape)

(833, 1, 48, 48)


# **CNN**

In [ ]:
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.metrics import categorical_accuracy

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Activation, Conv2D, Dropout
from keras.layers import Dense, BatchNormalization, MaxPooling2D , AveragePooling2D
from keras.models import Sequential, Model
from keras.layers import Input
from keras import layers
from keras.regularizers import l2

label_count = 7
batch_size = 32
epochs = 300
width, height = 48 , 48 

inputShape = (width,height,1)

regularizer = l2(0.01)

#base 
img_inp = Input(inputShape)

x = Conv2D(1, (3, 3), activation='relu', input_shape=(48, 48, 1), kernel_regularizer=l2(0.01))(img_inp)
x = Conv2D(64, (3, 3), padding='same',activation='relu') (x)
x = BatchNormalization() (x)
x = Conv2D(64, (3, 3), padding='same',activation='relu') (x)
x = BatchNormalization() (x)
x = MaxPooling2D(pool_size=(2,2), strides=(2, 2)) (x)
x = Dropout(0.5) (x)

x = Conv2D(128, (3, 3), padding='same', activation='relu') (x)
x = BatchNormalization() (x)
x = Conv2D(128, (3, 3), padding='same', activation='relu') (x)
x = BatchNormalization() (x)
x = Conv2D(128, (3, 3), padding='same', activation='relu') (x)
x = BatchNormalization() (x)
x = MaxPooling2D(pool_size=(2,2)) (x)
x = Dropout(0.5) (x)

x = Conv2D(256, (3, 3), padding='same', activation='relu') (x)
x = BatchNormalization() (x)
x = Conv2D(256, (3, 3), padding='same', activation='relu') (x)

x = Flatten() (x)

x = Dense(512, activation='relu') (x)
x = Dropout(0.5) (x)
x = Dense(256, activation='relu') (x)
x = Dropout(0.5) (x)
x = Dense(128, activation='relu') (x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu') (x)
x = Dropout(0.5)(x) 
x = Dense(7, activation='softmax')(x)

output = Dense(label_count, activation = 'softmax') (x)

# This is the model we will train
model = Model(inputs=img_inp, outputs=output)

optimizer = Adam(lr = 0.01)

model.compile(loss='categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['categorical_accuracy'])

model.summary()

filepath = "modelcp.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

trained = model.fit(X_train, y_train, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    verbose=1, 
                    validation_data=(np.array(X_valid), np.array(y_valid)), 
                    shuffle=True,
                    callbacks=callbacks_list)

print("****************************************************************************")
print("train completed")

Model: "model_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        (None, 48, 48, 1)         0         
_________________________________________________________________
conv2d_118 (Conv2D)          (None, 46, 46, 1)         10        
_________________________________________________________________
conv2d_119 (Conv2D)          (None, 46, 46, 64)        640       
_________________________________________________________________
batch_normalization_99 (Batc (None, 46, 46, 64)        256       
_________________________________________________________________
conv2d_120 (Conv2D)          (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization_100 (Bat (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 23, 23, 64)        0  

ValueError: ignored

# **Testing**

In [ ]:
from keras.preprocessing.image import img_to_array
import imutils
import cv2
from keras.models import load_model
import numpy as np
from keras.models import model_from_json

# parameters for loading data and images
detection_model_path = '/anaconda3/pkgs/libopencv-3.4.2-h7c891bd_1/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml' #gonna change
loaded_model = load_model("modelcp.h5")


# hyper-parameters for bounding boxes shape
# loading models
face_detection = cv2.CascadeClassifier(detection_model_path)
emotion_classifier = loaded_model
EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised",
 "neutral"]


#feelings_faces = []
#for index, emotion in enumerate(EMOTIONS):
   # feelings_faces.append(cv2.imread('emojis/' + emotion + '.png', -1))

# starting video streaming
cv2.namedWindow('your_face')
camera = cv2.VideoCapture(0)
while True:
    frame = camera.read()[1]
    #reading the frame
    frame = imutils.resize(frame,width=300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detection.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30),flags=cv2.CASCADE_SCALE_IMAGE)
    
    canvas = np.zeros((250, 300, 3), dtype="uint8")
    frameClone = frame.copy()
    if len(faces) > 0:
        faces = sorted(faces, reverse=True,
        key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = faces
                    # Extract the ROI of the face from the grayscale image, resize it to a fixed 28x28 pixels, and then prepare
            # the ROI for classification via the CNN
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (48,48))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
        
        preds = emotion_classifier.predict(roi)[0]
        emotion_probability = np.max(preds)
        label = EMOTIONS[preds.argmax()]

 
    for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
                # construct the label text
                text = "{}: {:.2f}%".format(emotion, prob * 100)

                # draw the label + probability bar on the canvas
               # emoji_face = feelings_faces[np.argmax(preds)]

                
                w = int(prob * 300)
                cv2.rectangle(canvas, (7, (i * 35) + 5),
                (w, (i * 35) + 35), (0, 0, 255), -1)
                cv2.putText(canvas, text, (10, (i * 35) + 23),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45,
                (255, 255, 255), 2)
                cv2.putText(frameClone, label, (fX, fY - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
                cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),
                              (0, 0, 255), 2)
#    for c in range(0, 3):
#        frame[200:320, 10:130, c] = emoji_face[:, :, c] * \
#        (emoji_face[:, :, 3] / 255.0) + frame[200:320,
#        10:130, c] * (1.0 - emoji_face[:, :, 3] / 255.0)


    cv2.imshow('your_face', frameClone)
    cv2.imshow("Probabilities", canvas)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()
